<a href="https://colab.research.google.com/github/dtoralg/IE_Calidad_ML/blob/main/Ejercicios/Modulo%202/Modulo_2_Ejercicio_5_Resuelto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 5: Creación de un Pipeline Avanzado de Machine Learning

## Objetivo
En este ejercicio, integrarás varios conceptos avanzados:
- Cargar y explorar el dataset Wine, disponible en scikit-learn.
- Preparar los datos para el modelado (definir variables, dividir en conjuntos de entrenamiento y prueba).
- Crear un pipeline que combine un preprocesador (StandardScaler) y un clasificador (por ejemplo, Logistic Regression).
- Realizar búsqueda de hiperparámetros mediante GridSearchCV.
- Evaluar el rendimiento del modelo mediante validación cruzada y métricas (exactitud y curva ROC).
- Visualizar la curva ROC del modelo final.

> **Pregunta de Reflexión:**  
> ¿Qué ventajas ofrece un pipeline para combinar el preprocesamiento y el modelado?  
> ¿Cómo mejora la búsqueda de hiperparámetros (GridSearchCV) el rendimiento del modelo?

¡Comencemos!


In [ ]:
# Paso 1: Importa las librerías necesarias y carga el dataset Wine.

# TODO: Importa pandas, numpy y las librerías de scikit-learn necesarias (datasets, model_selection, pipeline, preprocessing, etc.)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, accuracy_score

# Carga el dataset Wine usando load_wine con as_frame=True para obtener un DataFrame
wine = load_wine(as_frame=True)
df_wine = wine.frame

# Visualiza las primeras filas del dataset
print("Primeras filas del dataset Wine:")
display(df_wine.head())

# Muestra información general y estadísticas básicas
print("Información del dataset Wine:")
display(df_wine.info())
print("Estadísticas descriptivas:")
display(df_wine.describe())


In [ ]:
# Paso 2: Prepara los datos para el modelado.

# TODO: Define X (características) e y (etiquetas) a partir del DataFrame.
# La etiqueta en el dataset Wine es la columna 'target'.
X = df_wine.drop(columns=['target'])
y = df_wine['target']

# Divide los datos en conjuntos de entrenamiento y prueba (80% / 20%).
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Tamaño de X_train:", X_train.shape)
print("Tamaño de X_test:", X_test.shape)


In [ ]:
# Paso 3: Construye un pipeline que integre preprocesamiento y modelado.

# TODO: Crea un pipeline que incluya:
#  - Una etapa 'scaler' usando StandardScaler.
#  - Una etapa 'clf' usando LogisticRegression (puedes usar el solver 'liblinear').
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(solver='liblinear'))
])

# Define un diccionario de parámetros para la búsqueda de hiperparámetros.
# Ejemplo: Ajusta el parámetro 'C' y el tipo de penalización para LogisticRegression.
param_grid = {
    'clf__C': [0.01, 0.1, 1, 10, 100],
    'clf__penalty': ['l1', 'l2']
}

# TODO: Configura y ejecuta GridSearchCV con 5-fold cross validation usando el pipeline.
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Mejores parámetros encontrados:")
print(grid_search.best_params_)
print("Mejor score de validación:", grid_search.best_score_)


In [ ]:
# Paso 4: Evalúa el modelo en el conjunto de prueba y visualiza la curva ROC.

# TODO: Predice las probabilidades en el conjunto de prueba usando grid_search.predict_proba().
y_proba = grid_search.predict_proba(X_test)

# Calcula la curva ROC.
# NOTA: Para la curva ROC en un problema multiclase, se puede elegir la clase 1 (si existe) o usar la estrategia One-vs-Rest.
# Aquí usaremos la probabilidad de la clase 1.
fpr, tpr, thresholds = roc_curve(y_test, y_proba[:, 1], pos_label=1)

# Calcula el AUC
roc_auc = auc(fpr, tpr)

# Visualiza la curva ROC.
plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, label=f'Curva ROC (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC del Modelo')
plt.legend(loc='lower right')
plt.show()

# TODO: Calcula la exactitud del modelo en el conjunto de prueba y muéstrala.
y_pred = grid_search.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Exactitud en el conjunto de prueba:", test_accuracy)


In [ ]:
# Paso 5: Sistema de testeo para validar el pipeline y la evaluación.

# Este bloque de código realizará las siguientes comprobaciones:
# 1. Que X_train contenga registros.
# 2. Que el pipeline incluya las etapas 'scaler' y 'clf'.
# 3. Que se hayan encontrado mejores parámetros (grid_search.best_params_ no sea None).
# 4. Que la exactitud en el conjunto de prueba sea superior a 0.7 (valor razonable para este dataset).

# Test 1: Verificar que X_train no esté vacío.
assert X_train.shape[0] > 0, "Test fallido: X_train no contiene registros."

# Test 2: Verificar que el pipeline contenga las etapas 'scaler' y 'clf'.
pipeline_steps = [name for name, _ in grid_search.best_estimator_.steps]
assert 'scaler' in pipeline_steps, "Test fallido: La etapa 'scaler' no está en el pipeline."
assert 'clf' in pipeline_steps, "Test fallido: La etapa 'clf' no está en el pipeline."

# Test 3: Verificar que se hayan encontrado mejores parámetros.
assert grid_search.best_params_ is not None, "Test fallido: No se encontraron mejores parámetros."

# Test 4: Verificar que la exactitud en el conjunto de prueba sea superior a 0.7.
assert test_accuracy > 0.7, f"Test fallido: La exactitud en el conjunto de prueba es demasiado baja ({test_accuracy})."

print("Todos los tests se han pasado correctamente. El pipeline y la evaluación se realizaron como se esperaba.")


## Reflexión Final

1. ¿Qué ventajas encuentras al utilizar un pipeline para integrar preprocesamiento y modelado?
2. ¿Cómo contribuye GridSearchCV a la mejora del rendimiento del modelo?
3. ¿Qué importancia tiene la visualización de la curva ROC en la evaluación de un modelo de clasificación?
4. ¿Podrías proponer otras métricas o técnicas de validación que complementen este análisis?

_Responde estas preguntas en una celda Markdown adicional o en un comentario._
